In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [3]:
df=pd.read_csv('fraud_detection_dataset.csv')
pd.set_option('display.max_columns', None)
df

,Transaction_ID,Customer_ID,Transaction_Amount,Transaction_Date,Merchant_ID,Merchant_Category,Card_Type,Is_Online,Is_Fraud,Customer_Age,Customer_Gender,Customer_Income,Transaction_City,Device_Type,Is_International
0,T0001,C001,1878.955193,2022-01-01 00:00:00,M001,Entertainment,Credit,0.0,0.0,59.0,Male,NaN,NaN,Mobile,0.0
1,T0002,C002,4754.064389,2022-01-02 00:00:00,NaN,Clothing,Debit,0.0,0.0,66.0,Male,106204.26510,Houston,NaN,1.0
2,T0003,C003,3662.649770,2022-01-03 00:00:00,M003,Electronics,Prepaid,0.0,0.0,NaN,Female,104454.93990,Chicago,Tablet,0.0
3,T0004,C004,2997.305836,2022-01-04 00:00:00,M004,Entertainment,Credit,0.0,0.0,48.0,Male,51910.04732,New York,Tablet,0.0
4,T0005,NaN,788.533016,2022-01-05 00:00:00,M005,NaN,Credit,0.0,0.0,NaN,Female,102891.54740,Phoenix,Tablet,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,T0196,C046,1752.555777,2022-07-15 00:00:00,NaN,Clothing,Debit,1.0,NaN,63.0,Female,115086.35620,Los Angeles,Mobile,0.0
196,T0197,NaN,NaN,2022-07-16 00:00:00,M017,NaN,Credit,0.0,0.0,40.0,NaN,35105.78918,Chicago,Tablet,NaN
197,T0198,C048,NaN,2022-07-17 00:00:00,M018,Entertainment,NaN,0.0,1.0,49.0,Male,63233.48010,Phoenix,Tablet,0.0
198,T0199,NaN,4436.561257,2022-07-18 00:00:00,M019,Clothing,Prepaid,0.0,0.0,34.0,Male,114361.59200,Phoenix,Tablet,0.0


The following function will show me the top 5 rows in the dataset

In [4]:
df.head()

,Transaction_ID,Customer_ID,Transaction_Amount,Transaction_Date,Merchant_ID,Merchant_Category,Card_Type,Is_Online,Is_Fraud,Customer_Age,Customer_Gender,Customer_Income,Transaction_City,Device_Type,Is_International
0,T0001,C001,1878.955193,2022-01-01 00:00:00,M001,Entertainment,Credit,0.0,0.0,59.0,Male,NaN,NaN,Mobile,0.0
1,T0002,C002,4754.064389,2022-01-02 00:00:00,NaN,Clothing,Debit,0.0,0.0,66.0,Male,106204.26510,Houston,NaN,1.0
2,T0003,C003,3662.649770,2022-01-03 00:00:00,M003,Electronics,Prepaid,0.0,0.0,NaN,Female,104454.93990,Chicago,Tablet,0.0
3,T0004,C004,2997.305836,2022-01-04 00:00:00,M004,Entertainment,Credit,0.0,0.0,48.0,Male,51910.04732,New York,Tablet,0.0
4,T0005,NaN,788.533016,2022-01-05 00:00:00,M005,NaN,Credit,0.0,0.0,NaN,Female,102891.54740,Phoenix,Tablet,0.0


The following function will show me the last 5 rows in the dataset

## Dataframe Atributes

### Characterisation of the data set:

### Lets now visualize the data types of each column and any missing values

I will now inspect the data types of the columns

In [5]:
print(df.dtypes)

Transaction_ID         object
Customer_ID            object
Transaction_Amount    float64
Transaction_Date       object
Merchant_ID            object
Merchant_Category      object
Card_Type              object
Is_Online             float64
Is_Fraud              float64
Customer_Age          float64
Customer_Gender        object
Customer_Income       float64
Transaction_City       object
Device_Type            object
Is_International      float64
dtype: object


In [6]:
df.dtypes.value_counts()

object     9
float64    6
dtype: int64

Checking the shape of the dataframe

The following function, will display the number of rows (200) and the number of columns (15)

In [8]:
print('Shape of DataFrame (Observations and Rows) = ', df.shape)

Shape of DataFrame (Observations and Rows) =  (200, 15)


Now I will see the column names

In [9]:
df.columns

Index(['Transaction_ID', 'Customer_ID', 'Transaction_Amount',
       'Transaction_Date', 'Merchant_ID', 'Merchant_Category', 'Card_Type',
       'Is_Online', 'Is_Fraud', 'Customer_Age', 'Customer_Gender',
       'Customer_Income', 'Transaction_City', 'Device_Type',
       'Is_International'],
      dtype='object')

In [10]:
values = df.values
values

array([['T0001', 'C001', 1878.955193, ..., nan, 'Mobile', 0.0],
       ['T0002', 'C002', 4754.064389, ..., 'Houston', nan, 1.0],
       ['T0003', 'C003', 3662.64977, ..., 'Chicago', 'Tablet', 0.0],
       ...,
       ['T0198', 'C048', nan, ..., 'Phoenix', 'Tablet', 0.0],
       ['T0199', nan, 4436.561257, ..., 'Phoenix', 'Tablet', 0.0],
       ['T0200', 'C050', nan, ..., 'New York', 'Tablet', 0.0]],
      dtype=object)

### Number of distinct values in each feature

Lets now visualize the number of distinct values in each feature

In [11]:
for col in df.columns:
    print(col, ': Number of Distinct Values: ', len(df[col].unique()))

Transaction_ID : Number of Distinct Values:  200
Customer_ID : Number of Distinct Values:  51
Transaction_Amount : Number of Distinct Values:  163
Transaction_Date : Number of Distinct Values:  200
Merchant_ID : Number of Distinct Values:  31
Merchant_Category : Number of Distinct Values:  6
Card_Type : Number of Distinct Values:  4
Is_Online : Number of Distinct Values:  3
Is_Fraud : Number of Distinct Values:  3
Customer_Age : Number of Distinct Values:  51
Customer_Gender : Number of Distinct Values:  3
Customer_Income : Number of Distinct Values:  181
Transaction_City : Number of Distinct Values:  6
Device_Type : Number of Distinct Values:  4
Is_International : Number of Distinct Values:  3


Print the distinct values in each feature

In [12]:
for col in df:
    print('COLUMN NAME: ', col,':')
    print(df[col].unique())
    print('\n')

COLUMN NAME:  Transaction_ID :
['T0001' 'T0002' 'T0003' 'T0004' 'T0005' 'T0006' 'T0007' 'T0008' 'T0009'
 'T0010' 'T0011' 'T0012' 'T0013' 'T0014' 'T0015' 'T0016' 'T0017' 'T0018'
 'T0019' 'T0020' 'T0021' 'T0022' 'T0023' 'T0024' 'T0025' 'T0026' 'T0027'
 'T0028' 'T0029' 'T0030' 'T0031' 'T0032' 'T0033' 'T0034' 'T0035' 'T0036'
 'T0037' 'T0038' 'T0039' 'T0040' 'T0041' 'T0042' 'T0043' 'T0044' 'T0045'
 'T0046' 'T0047' 'T0048' 'T0049' 'T0050' 'T0051' 'T0052' 'T0053' 'T0054'
 'T0055' 'T0056' 'T0057' 'T0058' 'T0059' 'T0060' 'T0061' 'T0062' 'T0063'
 'T0064' 'T0065' 'T0066' 'T0067' 'T0068' 'T0069' 'T0070' 'T0071' 'T0072'
 'T0073' 'T0074' 'T0075' 'T0076' 'T0077' 'T0078' 'T0079' 'T0080' 'T0081'
 'T0082' 'T0083' 'T0084' 'T0085' 'T0086' 'T0087' 'T0088' 'T0089' 'T0090'
 'T0091' 'T0092' 'T0093' 'T0094' 'T0095' 'T0096' 'T0097' 'T0098' 'T0099'
 'T0100' 'T0101' 'T0102' 'T0103' 'T0104' 'T0105' 'T0106' 'T0107' 'T0108'
 'T0109' 'T0110' 'T0111' 'T0112' 'T0113' 'T0114' 'T0115' 'T0116' 'T0117'
 'T0118' 'T0119' 'T0

I will now look for duplicate rows

In [13]:
df.count()

Transaction_ID        200
Customer_ID           163
Transaction_Amount    162
Transaction_Date      200
Merchant_ID           160
Merchant_Category     162
Card_Type             162
Is_Online             180
Is_Fraud              180
Customer_Age          180
Customer_Gender       180
Customer_Income       180
Transaction_City      180
Device_Type           180
Is_International      180
dtype: int64

In [14]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

number of duplicate rows:  (0, 15)


In [15]:
df.duplicated().sum()

0

There is no duplicate data

The following function will give me a summary of statistics pertaining to the float columns and calculate the 5 number summary giving me the Mean, Standard Deviation, 1st Quartile: 25%, 3rd Quartile: 75%, Minimum: Smallest Observation in the Sample and Maximum: Largest Observation in the sample.

I will now call the .describe() function to quickly analyse the summary statistics for the numerical features.

In [16]:
df.describe()

,Transaction_Amount,Is_Online,Is_Fraud,Customer_Age,Customer_Income,Is_International
count,162.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,2483.627917,0.461111,0.083333,44.877778,72922.291160,0.111111
std,1460.796088,0.499876,0.277156,15.260002,30005.132560,0.315146
min,92.773266,0.000000,0.000000,18.000000,20638.587170,0.000000
25%,1226.883909,0.000000,0.000000,35.000000,45979.113178,0.000000
50%,2504.819185,0.000000,0.000000,45.000000,77290.908590,0.000000
75%,3844.424019,1.000000,0.000000,57.250000,99546.366692,0.000000
max,4934.565814,1.000000,1.000000,69.000000,119793.411100,1.000000


In [17]:
df.describe(include=object)

,Transaction_ID,Customer_ID,Transaction_Date,Merchant_ID,Merchant_Category,Card_Type,Customer_Gender,Transaction_City,Device_Type
count,200,163,200,160,162,162,180,180,180
unique,200,50,200,30,5,3,2,5,3
top,T0001,C001,2022-01-01 00:00:00,M005,Entertainment,Credit,Male,Houston,Mobile
freq,1,4,1,7,34,69,100,41,66


In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Transaction_Amount,162.0,2483.627917,1460.796088,92.773266,1226.883909,2504.819185,3844.424019,4934.565814
Is_Online,180.0,0.461111,0.499876,0.000000,0.000000,0.000000,1.000000,1.000000
Is_Fraud,180.0,0.083333,0.277156,0.000000,0.000000,0.000000,0.000000,1.000000
Customer_Age,180.0,44.877778,15.260002,18.000000,35.000000,45.000000,57.250000,69.000000
Customer_Income,180.0,72922.291160,30005.132560,20638.587170,45979.113178,77290.908590,99546.366692,119793.411100
Is_International,180.0,0.111111,0.315146,0.000000,0.000000,0.000000,0.000000,1.000000


In [19]:
print('length of data is', len(df))

length of data is 200


In [20]:
numerical_summary = df.describe().transpose()

palette = sns.color_palette("viridis", as_cmap=True)

numerical_summary.style.background_gradient(cmap=palette)

,count,mean,std,min,25%,50%,75%,max
Transaction_Amount,162.000000,2483.627917,1460.796088,92.773266,1226.883909,2504.819185,3844.424019,4934.565814
Is_Online,180.000000,0.461111,0.499876,0.000000,0.000000,0.000000,1.000000,1.000000
Is_Fraud,180.000000,0.083333,0.277156,0.000000,0.000000,0.000000,0.000000,1.000000
Customer_Age,180.000000,44.877778,15.260002,18.000000,35.000000,45.000000,57.250000,69.000000
Customer_Income,180.000000,72922.291160,30005.132560,20638.587170,45979.113178,77290.908590,99546.366692,119793.411100
Is_International,180.000000,0.111111,0.315146,0.000000,0.000000,0.000000,0.000000,1.000000
